In [54]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/test_features.csv


In [55]:
!rm train_targets_folds.csv
!wget https://github.com/Mayukhdeb/moa-classification-with-model-blending/raw/main/folds/train_targets_folds.csv

rm: cannot remove 'train_targets_folds.csv': No such file or directory
--2020-10-05 17:04:34--  https://github.com/Mayukhdeb/moa-classification-with-model-blending/raw/main/folds/train_targets_folds.csv
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Mayukhdeb/moa-classification-with-model-blending/main/folds/train_targets_folds.csv [following]
--2020-10-05 17:04:35--  https://raw.githubusercontent.com/Mayukhdeb/moa-classification-with-model-blending/main/folds/train_targets_folds.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.64.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.64.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10182351 (9.7M) [text/plain]
Saving to: ‘train_targets_folds.csv’

train_targets_folds 100%[=============

In [56]:
!pip install iterative-stratification

In [57]:
pd.read_csv("./train_targets_folds.csv").head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,kfold
0,id_95029ba69,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1,id_937bf8e8f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
2,id_59e0b53af,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
3,id_1b0b65599,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,id_1d650ed9d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [58]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler

import numpy as np 
import pandas as pd 

from IPython.display import clear_output
import random 
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter1d   ## smoother
from tqdm.notebook import tqdm, tnrange
import os
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

plt.rcParams['figure.figsize'] = 15, 7

CGREEN  = '\33[32m'
CBLUE =  '\033[34m'
CRED = '\033[1;31m'
CEND  = '\33[0m'

def seed_everything(seed=1903):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    print("seed kar diya")

In [59]:
if torch.cuda.is_available():
    device='cuda'
else:
    device='cpu'
    
device


'cuda'

In [60]:

def preprocess(df, select_features):
    df = df.copy()
    top_features = [ 0, 1, 2,   3,   4,   5,   6,   7,   9,  11,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  29,  30,  31,
        32,  33,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  58,  59,  60,
        61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
        74,  75,  76,  78,  79,  80,  81,  82,  83,  84,  86,  87,  88,
        89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
       115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197,
       198, 199, 200, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 217, 218, 219, 220, 221, 223, 224, 225, 226,
       227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
       240, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253,
       254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266,
       267, 268, 269, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280,
       281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 294,
       295, 296, 298, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,
       310, 311, 312, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323,
       324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336,
       337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349,
       350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362,
       363, 364, 365, 366, 367, 368, 369, 370, 371, 374, 375, 376, 377,
       378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 390, 391,
       392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404,
       405, 406, 407, 408, 409, 411, 412, 413, 414, 415, 416, 417, 418,
       419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431,
       432, 434, 435, 436, 437, 438, 439, 440, 442, 443, 444, 445, 446,
       447, 448, 449, 450, 453, 454, 456, 457, 458, 459, 460, 461, 462,
       463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475,
       476, 477, 478, 479, 481, 482, 483, 484, 485, 486, 487, 488, 489,
       490, 491, 492, 493, 494, 495, 496, 498, 500, 501, 502, 503, 505,
       506, 507, 509, 510, 511, 512, 513, 514, 515, 518, 519, 520, 521,
       522, 523, 524, 525, 526, 527, 528, 530, 531, 532, 534, 535, 536,
       538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 549, 550, 551,
       552, 554, 557, 559, 560, 561, 562, 565, 566, 567, 568, 569, 570,
       571, 572, 573, 574, 575, 577, 578, 580, 581, 582, 583, 584, 585,
       586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 599,
       600, 601, 602, 606, 607, 608, 609, 611, 612, 613, 615, 616, 617,
       618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630,
       631, 632, 633, 634, 635, 636, 637, 638, 639, 641, 642, 643, 644,
       645, 646, 647, 648, 649, 650, 651, 652, 654, 655, 656, 658, 659,
       660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672,
       673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685,
       686, 687, 688, 689, 691, 692, 693, 694, 695, 696, 697, 699, 700,
       701, 702, 704, 705, 707, 708, 709, 710, 711, 713, 714, 716, 717,
       718, 720, 721, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732,
       733, 734, 735, 737, 738, 739, 740, 742, 743, 744, 745, 746, 747,
       748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 759, 760, 761,
       762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774,
       775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788,
       789, 790, 792, 793, 794, 795, 796, 797, 798, 800, 801, 802, 803,
       804, 805, 806, 808, 809, 811, 813, 814, 815, 816, 817, 818, 819,
       821, 822, 823, 825, 826, 827, 828, 829, 830, 831, 832, 834, 835,
       837, 838, 839, 840, 841, 842, 845, 846, 847, 848, 850, 851, 852,
       854, 855, 856, 858, 859, 860, 861, 862, 864, 866, 867, 868, 869,
       870, 871, 872, 873, 874]
    all_columns = df.columns

    if select_features == True:
        df=df[all_columns[top_features]]
    else:
        pass
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    df.loc[:, 'cp_time'] = df.loc[:, 'cp_time'].map({ 24: 0, 48: 1, 72: 2})
    
    return df

In [61]:
class MoaDataset(Dataset):
    def __init__(self,features,targets):
        
        self.features  = features
        self.targets = targets
        
    def sizes(self):
        print("features size = ", self.features.shape[1])
        print("targets size = ", self.targets.shape[1])
        
    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, item):
        return {
            "x": torch.tensor(self.features[item, :], dtype=torch.float),
            "y": torch.tensor(self.targets[item, :], dtype=torch.float)
        }

In [62]:

class Engine:
      def __init__(self, model, optimizer, device, scheduler):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler

      @staticmethod
      def loss_fn(targets, outputs):
        return nn.BCEWithLogitsLoss()(outputs, targets)

      def train(self, data_loader):
        self.model.train()
        final_loss=0
        for data in data_loader:
            self.optimizer.zero_grad()
            inputs = data["x"].to(self.device)
            targets = data["y"].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(targets, outputs)
            loss.backward()
            
            self.optimizer.step()
            
            final_loss += loss.item()
        self.scheduler.step(1.)
        return(final_loss / len(data_loader))


      def evaluate(self, data_loader):
        self.model.train()
        final_loss=0
        for data in data_loader:
            #self.optimizer.zero_grad()
            inputs = data["x"].to(self.device)
            targets = data["y"].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(targets, outputs)
            #loss.backward()
            #self.optimizer.step()
            final_loss += loss.item()
        return(final_loss / len(data_loader))

In [77]:
def run_training(model_class ,fold, save_model=True, select_features = False, model_id = 0):
    EPOCHS = 100
    df = pd.read_csv('../input/lish-moa/train_features.csv')
    df = preprocess(df, select_features = select_features)
    targets_df = pd.read_csv('./train_targets_folds.csv')
    
    feature_columns = df.drop("sig_id", axis=1).columns
    target_columns = targets_df.drop(["sig_id", "kfold"], axis=1).columns
    

    df = df.merge(targets_df, on="sig_id", how="left")
    print(df.shape)
    df = df[df.kfold != "hold"]
    print(df.shape)

    train_df = df[df.kfold != str(fold)].reset_index(drop=True)
    valid_df = df[df.kfold == str(fold)].reset_index(drop=True)

    
    x_train = train_df[feature_columns].to_numpy()
    y_train = train_df[target_columns].to_numpy()
    
    x_val = valid_df[feature_columns].to_numpy()
    y_val = valid_df[target_columns].to_numpy()
    
    train_dataset = MoaDataset(features = x_train, targets=y_train)
    valid_dataset = MoaDataset(features = x_val, targets=y_val)
    
    train_loader = DataLoader(
        train_dataset, batch_size=128, shuffle=True
    )
    
    val_loader = DataLoader(
        valid_dataset, batch_size=1024, shuffle=True
    )
    
    model = model_class()
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), 
                                       lr = 0.004299882049752947, 
                                       weight_decay=2e-5)
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min', 
                                                 factor=0.1, ## wooo hoo
                                                 patience=7, ## was 3 for 158 
                                                 eps=1e-4, 
                                                 verbose=True)
    criterion = nn.BCEWithLogitsLoss()
    eng = Engine(model, optimizer, device='cuda', scheduler = scheduler)
    
    best_loss = 999
    early_stop_iter = 15
    early_stop_count=0

    for epoch in tnrange(EPOCHS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.evaluate(val_loader)
        #print("train_loss:", train_loss, "val_loss:", valid_loss)
        if valid_loss<best_loss:
            best_loss = valid_loss
            if save_model:
                torch.save(model.state_dict(), "model_"+str(fold + model_id) +".pth")
                pass
        else:
            early_stop_count +=1
            
        if early_stop_count>early_stop_iter:
            break
                    
    return(best_loss)


In [78]:

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(785)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(785, 2048))
        
        self.batch_norm2 = nn.BatchNorm1d(2048)
        self.dropout2 = nn.Dropout(0.5)
        self.dense2 = nn.utils.weight_norm(nn.Linear(2048, 1048))
        
        self.batch_norm3 = nn.BatchNorm1d(1048)
        self.dropout3 = nn.Dropout(0.5)
        self.dense3 = nn.utils.weight_norm(nn.Linear(1048, 206))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

In [ ]:
best_losses = []
num_folds = 7
for fold in range(0, num_folds):
    print(best_losses)
    best_losses.append(run_training(model_class = Model, fold=fold, select_features = True))

In [95]:

class Model_2(nn.Module):
    def __init__(self):
        super(Model_2, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(875)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(875, 2048))  ## was 948 and 2 layers ofr best yet 128
        
#         self.batch_norm2 = nn.BatchNorm1d(948)
#         self.dropout2 = nn.Dropout(0.5)
#         self.dense2 = nn.utils.weight_norm(nn.Linear(948, 948))
        
        self.batch_norm3 = nn.BatchNorm1d(2048)
        self.dropout3 = nn.Dropout(0.5)
        self.dense3 = nn.utils.weight_norm(nn.Linear(2048, 206))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.leaky_relu(self.dense1(x))
        
#         x = self.batch_norm2(x)
#         x = self.dropout2(x)
#         x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

In [96]:
best_losses_2 = []
num_folds = 7
for fold in range(num_folds):
    print(best_losses_2)
    best_losses_2.append(run_training(model_class = Model_2, fold=fold, select_features = False, model_id = 100))

[]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.015385986305773258]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.015385986305773258, 0.015402783950169882]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.015385986305773258, 0.015402783950169882, 0.01547326830526193]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.015385986305773258, 0.015402783950169882, 0.01547326830526193, 0.015219192020595074]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.015385986305773258, 0.015402783950169882, 0.01547326830526193, 0.015219192020595074, 0.015409963515897592]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.015385986305773258, 0.015402783950169882, 0.01547326830526193, 0.015219192020595074, 0.015409963515897592, 0.015432303150494894]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.



In [97]:
print(best_losses_2)  ## better than old model
np.array(best_losses_2).mean()

[0.015385986305773258, 0.015402783950169882, 0.01547326830526193, 0.015219192020595074, 0.015409963515897592, 0.015432303150494894, 0.015624086372554302]


0.015421083374392418

In [121]:
!zip -r models.zip *.pth 

updating: model_0.pth (deflated 5%)
updating: model_1.pth (deflated 4%)
updating: model_100.pth (deflated 6%)
updating: model_101.pth (deflated 6%)
updating: model_102.pth (deflated 6%)
updating: model_103.pth (deflated 6%)
updating: model_104.pth (deflated 6%)
updating: model_105.pth (deflated 6%)
updating: model_106.pth (deflated 6%)
updating: model_2.pth (deflated 4%)
updating: model_3.pth (deflated 4%)
updating: model_4.pth (deflated 4%)
updating: model_5.pth (deflated 5%)
updating: model_6.pth (deflated 5%)


In [87]:
def load_fold(fold, select_features = False):

    df = pd.read_csv('../input/lish-moa/train_features.csv')
    df = preprocess(df, select_features = select_features)
    targets_df = pd.read_csv('./train_targets_folds.csv')

    feature_columns = df.drop("sig_id", axis=1).columns
    target_columns = targets_df.drop(["sig_id", "kfold"], axis=1).columns


    df = df.merge(targets_df, on="sig_id", how="left")

    train_df = df[df.kfold != str(fold)].reset_index(drop=True)
    valid_df = df[df.kfold == str(fold)].reset_index(drop=True)

    x_train = train_df[feature_columns].to_numpy()
    y_train = train_df[target_columns].to_numpy()

    x_val = valid_df[feature_columns].to_numpy()
    y_val = valid_df[target_columns].to_numpy()

    return x_train, y_train, x_val, y_val

class blend():
    def __init__(self,all_preds_np):
        
        self.all_preds = all_preds_np
           
    def predict(self, weights):
        self.weights = weights
        final_pred = np.zeros_like(self.all_preds[0])
        
        for i in range(len(self.all_preds)):
            final_pred += self.all_preds[i] * self.weights[i]
            
        final_pred = final_pred/self.weights.sum()
        
        return final_pred


def inference_fn(model, test_features, device):

    results = np.zeros([test_features.shape[0], 206])
    test_features_tensor = torch.tensor(test_features).float().to(device)
    rows = model(test_features_tensor).sigmoid().detach().cpu().numpy()
    results = rows 

    return results 

class pytorch_model():
    def __init__(self, model_class, model_path, device):
        
        self.model_class = model_class
        self.model_path = model_path
        self.device = device
        
        self.model = self.model_class()
        if self.device == "cuda":
            self.model.load_state_dict(torch.load(self.model_path))
        else:
            self.model.load_state_dict(torch.load(self.model_path, map_location=torch.device('cpu') ))

        self.model.to(self.device)
        self.model.eval()
        
    def predict(self, x):
        
        pred = self.model(x)
        
        return pred 
    
    def __call__(self, x):
        return self.predict(x)


def log_loss_metric(y_true, y_pred):
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = - np.mean(np.mean(y_true * np.log(y_pred_clip) + (1 - y_true) * np.log(1 - y_pred_clip), axis = 1))
    return loss

In [119]:
x_train, y_train, x_val, y_val = load_fold("hold", select_features = True)

all_preds = []
for i in range(7):
    m1  = pytorch_model(Model, "model_"+ str(i) +".pth", device)
    p1 = inference_fn(m1, x_val, device)
    l1 = log_loss_metric(y_val, p1)
    print(l1)
    all_preds.append(p1)

0.015078569481878528
0.01505837406536361
0.015017550304499687
0.01505500348238407
0.015107810259518569
0.015075756093567884
0.015096678756626998


In [120]:
x_train, y_train, x_val, y_val = load_fold("hold", select_features = False)

for i in range(7):
    m2  = pytorch_model(Model_2, "model_"+ str(i+100) +".pth", device)
    p2 = inference_fn(m2, x_val, device)
    l2 = log_loss_metric(y_val, p2)
    print(l2)
    all_preds.append(p2)

0.015014940045815068
0.014990424248283948
0.014927199110675977
0.01496060591166882
0.015010664236536377
0.015055453028840583
0.014954357366795602


In [90]:
!pip install optuna

In [91]:
import optuna

In [113]:
def scale_data(data, lower, upper):
    
    scaled = np.interp(data, (data.min(), data.max()), (lower, upper))
    return scaled 

def find_loss(params):
    
    weights_np = np.array(list(params.values())[:-2])
    blend_boi = blend(all_preds)
    sample_infer = blend_boi.predict(weights_np)
    
    
    pred_scaled = scale_data(sample_infer, params["lower"], params["upper"])
    loss = log_loss_metric(y_val, pred_scaled)
    
    return loss


def objective(trial):

    params = {
        "w1": trial.suggest_uniform("w1", 0.0001, 1.0),
        "w2": trial.suggest_uniform("w2", 0.0001, 1.0),
        "w3": trial.suggest_uniform("w3", 0.0001, 1.0),
        "w4": trial.suggest_uniform("w4", 0.0001, 1.0),
        "w5": trial.suggest_uniform("w5", 0.0001, 1.0),
        "w6": trial.suggest_uniform("w6", 0.0001, 1.0),
        "w7": trial.suggest_uniform("w7", 0.0001, 1.0),
        "w8": trial.suggest_uniform("w8", 0.0001, 1.0),
        "w9": trial.suggest_uniform("w9", 0.0001, 1.0),
        "w10": trial.suggest_uniform("w10", 0.0001, 1.0),
        "w11": trial.suggest_uniform("w11", 0.0001, 1.0),
        "w12": trial.suggest_uniform("w12", 0.0001, 1.0),
        "w13": trial.suggest_uniform("w13", 0.0001, 1.0),
        "w14": trial.suggest_uniform("w14", 0.0001, 1.0),
        "upper": trial.suggest_uniform("upper", 0.5, 1.5),
        "lower": trial.suggest_uniform("lower", -0.5, 0.5),

    }

    loss_ = find_loss(params) 
    return loss_

In [114]:
study = optuna.create_study(direction="minimize")

[I 2020-10-05 18:24:02,948] A new study created in memory with name: no-name-bff735c6-05bd-4410-85ce-7b5839a31064


In [116]:
study.optimize(objective, n_trials=1000)

[I 2020-10-05 18:26:20,978] Trial 1000 finished with value: 0.030257645744467064 and parameters: {'w1': 0.365731100936601, 'w2': 0.4254899953837905, 'w3': 0.7223042341137498, 'w4': 0.47209786394711256, 'w5': 0.6621096545943855, 'w6': 0.7680056040441251, 'w7': 0.5264133862714091, 'w8': 0.53393759305929, 'w9': 0.6272358565240967, 'w10': 0.0981228745098153, 'w11': 0.6238979477979557, 'w12': 0.5076369576280736, 'w13': 0.5827749622484144, 'w14': 0.7948734471146671, 'upper': 1.0380079201796732, 'lower': 0.01825943878021475}. Best is trial 503 with value: 0.014752477557872073.
[I 2020-10-05 18:26:21,161] Trial 1001 finished with value: 0.08193628590528122 and parameters: {'w1': 0.06818051902218619, 'w2': 0.37774547113886137, 'w3': 0.7235075500638583, 'w4': 0.4867475054242968, 'w5': 0.5958923024326894, 'w6': 0.7321417267376821, 'w7': 0.35095009973823865, 'w8': 0.5310951512749525, 'w9': 0.6439275286139852, 'w10': 0.025743636208594545, 'w11': 0.7751727205789728, 'w12': 0.46061246622364715, 'w13'

[I 2020-10-05 18:26:23,633] Trial 1014 finished with value: 0.07160077888596632 and parameters: {'w1': 0.10229479622343807, 'w2': 0.5568553640865487, 'w3': 0.7030740277496845, 'w4': 0.5087603125284813, 'w5': 0.33289934432434143, 'w6': 0.735851076130324, 'w7': 0.536815459915374, 'w8': 0.5098773233140609, 'w9': 0.8733592327945235, 'w10': 0.03550657029715529, 'w11': 0.7457008521437738, 'w12': 0.40604992449300037, 'w13': 0.5360903924610403, 'w14': 0.2556305113684729, 'upper': 1.044443418599619, 'lower': -0.025010155463157456}. Best is trial 503 with value: 0.014752477557872073.
[I 2020-10-05 18:26:23,814] Trial 1015 finished with value: 0.023940601127049075 and parameters: {'w1': 0.0554251119979296, 'w2': 0.35171978031061335, 'w3': 0.671077173426871, 'w4': 0.44882250376769356, 'w5': 0.6047678627218932, 'w6': 0.6948347741417518, 'w7': 0.5735352503896295, 'w8': 0.47760643476990466, 'w9': 0.845560472804598, 'w10': 0.49617183320635694, 'w11': 0.7808851750835264, 'w12': 0.4874536951766186, 'w13

[I 2020-10-05 18:26:26,273] Trial 1028 finished with value: 0.03477718911307414 and parameters: {'w1': 0.029772080507802428, 'w2': 0.4316503641136461, 'w3': 0.7917750357727756, 'w4': 0.5424337438821516, 'w5': 0.870857109363421, 'w6': 0.7949152685143169, 'w7': 0.5839646407702038, 'w8': 0.4598219244049755, 'w9': 0.8709417807272598, 'w10': 0.36427271952566315, 'w11': 0.5774216842140971, 'w12': 0.5231166176496528, 'w13': 0.545251048407552, 'w14': 0.29983680856319733, 'upper': 1.2108090907178313, 'lower': 0.022431555077621613}. Best is trial 503 with value: 0.014752477557872073.
[I 2020-10-05 18:26:26,456] Trial 1029 finished with value: 0.08092598807493977 and parameters: {'w1': 0.1334340185450873, 'w2': 0.3991547517331382, 'w3': 0.7557445030749819, 'w4': 0.5147575356798846, 'w5': 0.8359180036618425, 'w6': 0.6877136982259463, 'w7': 0.5658596162710439, 'w8': 0.5826278462525574, 'w9': 0.852115575164246, 'w10': 0.38671088748140425, 'w11': 0.5493891699705303, 'w12': 0.5871954363315932, 'w13': 

[I 2020-10-05 18:26:28,959] Trial 1042 finished with value: 0.0791311637393942 and parameters: {'w1': 0.04799555557124664, 'w2': 0.42229274642461817, 'w3': 0.7070940572856215, 'w4': 0.5587143002705596, 'w5': 0.7943875737645807, 'w6': 0.7774446036708927, 'w7': 0.5164023612133268, 'w8': 0.43195044882034117, 'w9': 0.8634695990517485, 'w10': 0.35217639650782706, 'w11': 0.5602989559245508, 'w12': 0.5117896025058112, 'w13': 0.5573223351579206, 'w14': 0.25492062177552866, 'upper': 1.1581379730074486, 'lower': -0.04529074642564909}. Best is trial 503 with value: 0.014752477557872073.
[I 2020-10-05 18:26:29,189] Trial 1043 finished with value: 0.0360888359121833 and parameters: {'w1': 0.9345289951223335, 'w2': 0.43710907050854414, 'w3': 0.7349446999560123, 'w4': 0.5353945996422501, 'w5': 0.5775286725547715, 'w6': 0.7232418261384854, 'w7': 0.5966953619252381, 'w8': 0.5492022993319567, 'w9': 0.804957766514975, 'w10': 0.41948470895355766, 'w11': 0.5892530024654283, 'w12': 0.4860754328072719, 'w13'

[I 2020-10-05 18:26:32,044] Trial 1056 finished with value: 0.026085480266969722 and parameters: {'w1': 0.14457641126838627, 'w2': 0.4880141849710138, 'w3': 0.7818574250851468, 'w4': 0.4832433489688166, 'w5': 0.8208464325390603, 'w6': 0.6546047038174924, 'w7': 0.5219403762172895, 'w8': 0.47289962314358003, 'w9': 0.854834585622204, 'w10': 0.3246360553272027, 'w11': 0.7215726625403113, 'w12': 0.5230445823956578, 'w13': 0.633248152971711, 'w14': 0.8605935127737057, 'upper': 1.2096612022448732, 'lower': 0.013047171251279518}. Best is trial 503 with value: 0.014752477557872073.
[I 2020-10-05 18:26:32,230] Trial 1057 finished with value: 0.07402686578065913 and parameters: {'w1': 0.27897089077387205, 'w2': 0.4672789750079493, 'w3': 0.7335946211684004, 'w4': 0.4559586939923997, 'w5': 0.8251410900963341, 'w6': 0.706224829897963, 'w7': 0.5661318352294367, 'w8': 0.4521649351873236, 'w9': 0.8175502632541913, 'w10': 0.35755966150978014, 'w11': 0.6470172986667749, 'w12': 0.5690291277129482, 'w13': 

...

[I 2020-10-05 18:29:57,793] Trial 1938 finished with value: 0.020005888453295543 and parameters: {'w1': 0.6045878561865932, 'w2': 0.3607582275744723, 'w3': 0.3006275194432489, 'w4': 0.11195481124259322, 'w5': 0.3451260621706291, 'w6': 0.8254742223336835, 'w7': 0.6028834926857544, 'w8': 0.41062633900300605, 'w9': 0.8422588244566526, 'w10': 0.05805481196900676, 'w11': 0.7116773275985333, 'w12': 0.5992380642598555, 'w13': 0.7124489526993438, 'w14': 0.8941923674377147, 'upper': 1.0483427651975565, 'lower': 0.006719772852691758}. Best is trial 503 with value: 0.014752477557872073.
[I 2020-10-05 18:29:58,137] Trial 1939 finished with value: 0.06512410874686332 and parameters: {'w1': 0.08033401970706466, 'w2': 0.3458048838785707, 'w3': 0.3081477451502689, 'w4': 0.49405285170191304, 'w5': 0.6550405201500761, 'w6': 0.8845602059103406, 'w7': 0.07177473236386284, 'w8': 0.428929770414741, 'w9': 0.892194856580184, 'w10': 0.05465032080463003, 'w11': 0.6707194559692284, 'w12': 0.4745951070505863, 'w1

[I 2020-10-05 18:30:01,955] Trial 1952 finished with value: 0.06573919263092337 and parameters: {'w1': 0.6373855698844981, 'w2': 0.40731917805176654, 'w3': 0.3241141644304173, 'w4': 0.43207660171080525, 'w5': 0.659493513349811, 'w6': 0.8523183803430535, 'w7': 0.6414251913408096, 'w8': 0.44691735763844387, 'w9': 0.8222093198466746, 'w10': 0.2561682180077986, 'w11': 0.6610676030305593, 'w12': 0.4897010278833196, 'w13': 0.6510994111039955, 'w14': 0.44964232597799975, 'upper': 1.0300632350706447, 'lower': 0.05449055819848483}. Best is trial 503 with value: 0.014752477557872073.
[I 2020-10-05 18:30:02,233] Trial 1953 finished with value: 0.01568992916879894 and parameters: {'w1': 0.07503192443154, 'w2': 0.4279562360621067, 'w3': 0.3753376843552162, 'w4': 0.49387091820211004, 'w5': 0.9997975275439581, 'w6': 0.8121128421173519, 'w7': 0.6514581975843998, 'w8': 0.39781036170564243, 'w9': 0.8780696582124535, 'w10': 0.019467218876772954, 'w11': 0.6880253027976369, 'w12': 0.5251153949753138, 'w13'

[I 2020-10-05 18:30:05,740] Trial 1966 finished with value: 0.018330457774489627 and parameters: {'w1': 0.09581449712183297, 'w2': 0.4118374985181964, 'w3': 0.9330507338470928, 'w4': 0.6999585827290952, 'w5': 0.879012915830554, 'w6': 0.8515718319147905, 'w7': 0.5701006698945423, 'w8': 0.46051708034783523, 'w9': 0.899300347209863, 'w10': 0.28397933107944034, 'w11': 0.722444198902828, 'w12': 0.5160619656092725, 'w13': 0.6403988583599871, 'w14': 0.9000950419105355, 'upper': 1.0726013459578436, 'lower': 0.004647356724960266}. Best is trial 503 with value: 0.014752477557872073.
[I 2020-10-05 18:30:06,003] Trial 1967 finished with value: 0.06107188330139506 and parameters: {'w1': 0.07140555449378136, 'w2': 0.4360125112761553, 'w3': 0.2713472954966859, 'w4': 0.5231507477143068, 'w5': 0.4816895524890637, 'w6': 0.8159436091521924, 'w7': 0.4774137431984705, 'w8': 0.4131134830078847, 'w9': 0.5930292336103876, 'w10': 0.27802466660520514, 'w11': 0.7642747423171834, 'w12': 0.48470067647009274, 'w13'

[I 2020-10-05 18:30:09,937] Trial 1980 finished with value: 0.019698182208482802 and parameters: {'w1': 0.043251622836846075, 'w2': 0.46224799359997143, 'w3': 0.55878428934111, 'w4': 0.40355534551793193, 'w5': 0.4328331475815214, 'w6': 0.857287023499262, 'w7': 0.6907373377050519, 'w8': 0.3696355023088765, 'w9': 0.8619920639187667, 'w10': 0.10799668603933166, 'w11': 0.7567797852297621, 'w12': 0.5156142299719016, 'w13': 0.7545259698836215, 'w14': 0.9225209410099182, 'upper': 1.0405432670056902, 'lower': 0.006421266154539433}. Best is trial 503 with value: 0.014752477557872073.
[I 2020-10-05 18:30:10,200] Trial 1981 finished with value: 0.07710517321340751 and parameters: {'w1': 0.11791204796667529, 'w2': 0.39206845711683536, 'w3': 0.3173691846433364, 'w4': 0.5000166072570384, 'w5': 0.6535008931834341, 'w6': 0.7652927129132922, 'w7': 0.5424308169839375, 'w8': 0.4304798457450967, 'w9': 0.8097949410020865, 'w10': 0.03656231533131804, 'w11': 0.9409435557495882, 'w12': 0.5022755549701717, 'w1

[I 2020-10-05 18:30:13,649] Trial 1994 finished with value: 0.019960758401445518 and parameters: {'w1': 0.632359725745239, 'w2': 0.9649641627819796, 'w3': 0.33604337302049914, 'w4': 0.533149240118432, 'w5': 0.3510383446239122, 'w6': 0.7892785788430351, 'w7': 0.6240002502868505, 'w8': 0.4835323891436696, 'w9': 0.7783448204220882, 'w10': 0.3231155943064913, 'w11': 0.7688700878535082, 'w12': 0.45201693704981516, 'w13': 0.5005923365828305, 'w14': 0.30576395267016954, 'upper': 1.06629089356371, 'lower': 0.0066261977043527995}. Best is trial 503 with value: 0.014752477557872073.
[I 2020-10-05 18:30:13,908] Trial 1995 finished with value: 0.07190652860023594 and parameters: {'w1': 0.1042681868357754, 'w2': 0.5600567145149046, 'w3': 0.2868639636452768, 'w4': 0.7176560459679763, 'w5': 0.5834817953996505, 'w6': 0.7242414103079041, 'w7': 0.6093181781001888, 'w8': 0.3762646500314842, 'w9': 0.9299878238043857, 'w10': 0.23836587643064838, 'w11': 0.7027708487592275, 'w12': 0.4979684024153144, 'w13': 

In [117]:
best_trial = study.best_params
best_trial

{'w1': 0.5377486890164793,
 'w2': 0.5097100875436078,
 'w3': 0.6692194393318107,
 'w4': 0.5688380454620098,
 'w5': 0.3576010694445187,
 'w6': 0.7386344176389785,
 'w7': 0.14518595812555402,
 'w8': 0.4874201084138915,
 'w9': 0.8159252562826513,
 'w10': 0.35415573755787044,
 'w11': 0.7215591959995986,
 'w12': 0.37577591320917025,
 'w13': 0.5665083783714475,
 'w14': 0.9271972682025718,
 'upper': 0.9997642802264908,
 'lower': -0.00039162782951547695}

In [118]:
x_train, y_train, x_val, y_val = load_fold("hold", select_features = True)

blend_boi = blend(all_preds)
noice_weights = np.array(list(best_trial.values())[:-2])
sample_infer = blend_boi.predict(noice_weights)
sample_infer = scale_data(sample_infer, best_trial["lower"], best_trial["upper"])
loss = log_loss_metric(y_val, sample_infer)
print("loss on hold set: ", loss)

loss on hold set:  0.014752477557872073


In [71]:
x_train, y_train, x_val, y_val = load_fold("hold", select_features = True)

blend_boi = blend(all_preds)
noice_weights = np.array(list(best_trial.values())[:-2])
sample_infer = blend_boi.predict(noice_weights)
loss = log_loss_metric(y_val, sample_infer)
print("loss on hold set: ", loss)

loss on hold set:  0.014954230654160877


In [123]:
list(best_trial.values())

[0.5377486890164793,
 0.5097100875436078,
 0.6692194393318107,
 0.5688380454620098,
 0.3576010694445187,
 0.7386344176389785,
 0.14518595812555402,
 0.4874201084138915,
 0.8159252562826513,
 0.35415573755787044,
 0.7215591959995986,
 0.37577591320917025,
 0.5665083783714475,
 0.9271972682025718,
 0.9997642802264908,
 -0.00039162782951547695]

In [124]:
df_w = pd.DataFrame(
    np.array(list(best_trial.values())).reshape(1,-1), 
    columns = list(best_trial.keys())
)

In [125]:
df_w.to_csv("blend_weights.csv", index = False)

In [ ]:
df_w